<a href="https://colab.research.google.com/github/TLiu277/speech-recognition/blob/main/simple_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForCausalLM


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 11.5 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 42.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import datetime
pre_model = 'microsoft/DialoGPT-large'

tokenizer = AutoTokenizer.from_pretrained(pre_model)
model = AutoModelForCausalLM.from_pretrained(pre_model)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
user_message = 'Hello'

# encode the user message
# 'eos_token' stands for the end of sentence token
user_input_ids = tokenizer.encode(user_message + tokenizer.eos_token, return_tensors='pt')

# generated a response by the model
bot_output_ids = model.generate(user_input_ids, pad_token_id=tokenizer.eos_token_id)

# decode previous message to text
start_of_bot_message = user_input_ids.shape[-1]
bot_message = tokenizer.decode(bot_output_ids[:, start_of_bot_message:][0], skip_special_tokens=True)

# prints the decoded message
print(bot_message)

Hiya


In [ ]:
class Feedback:
  def __init__(self, pre_model = 'microsoft/DialoGPT-large'):
      self.tokenizer,self.model = self.load_model(pre_model)

  def load_model(self, pre_model):
      tokenizer = AutoTokenizer.from_pretrained(pre_model)
      model = AutoModelForCausalLM.from_pretrained(pre_model)
      return tokenizer, model

  def get_reply(self, user_message):
      # save message from the user
      self.chat_history.append({
        'text':user_message, 
        'time':str(datetime.datetime.now().time().replace(microsecond=0))
      })

      # encode the user message
      user_input_ids = self.tokenizer.encode(user_message + self.tokenizer.eos_token, return_tensors='pt')

      # append the encoded message to the past history so the model is aware of past context
      if self.chat_history_ids is not None:
        user_input_ids = torch.cat([self.chat_history_ids, user_input_ids], dim=-1)

      # generated a response by the model
      bot_output_ids = self.model.generate(user_input_ids, pad_token_id=self.tokenizer.eos_token_id,do_sample=True, 
      max_length=1000, 
      top_k=100, 
      top_p=0.95,
      temperature=0.8,)
      # decode previous message to text
      start_of_bot_message = user_input_ids.shape[-1]
      bot_message = tokenizer.decode(bot_output_ids[:, start_of_bot_message:][0], skip_special_tokens=True)
       
      # prints the decoded message
      return bot_message

In [ ]:
Feedback()